In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import time
import os
from PIL import Image
from keras.layers import Dense,Conv2D,Dropout,Dense,Flatten,Conv2DTranspose,BatchNormalization,LeakyReLU,Reshape,Activation

In [ ]:
IMG_SIZE = 32
RANDOM_SEED=100
DATA_PATH = '/kaggle/input/celeba-dataset/img_align_celeba/img_align_celeba/'
EPOCHS = 20
BUFFER_SIZE = 60000
BATCH_SIZE = 128

PREVIEW_ROWS = 4
PREVIEW_COLS = 7
PREVIEW_MARGIN = 16

training_data=[]
for dirname, _, filenames in os.walk(DATA_PATH):
    for filename in filenames:
        training_data.append(DATA_PATH+filename)

In [ ]:
print(training_data[:10])
len(training_data)

In [ ]:
images=[]
for path in training_data[:50000]:
    image=Image.open(path).resize((IMG_SIZE, IMG_SIZE))
    images.append(np.asarray(image))
images = np.reshape(images,(-1,IMG_SIZE,
            IMG_SIZE,3))
images = images.astype(np.float32)
images = (images - 127.5) / 127.5 
print(images[:10])

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices(images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [ ]:
def make_generator_model():
    model = tf.keras.Sequential()
    
    model.add(Dense(4*4*512,input_shape=(RANDOM_SEED,)))
    model.add(Reshape((4,4,512)))
    
    model.add(Conv2DTranspose(256,kernel_size=4,padding="same",strides=2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    
    model.add(Conv2DTranspose(128,kernel_size=4,padding="same",strides=2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    
    model.add(Conv2DTranspose(3,kernel_size=4,padding="same",strides=2))
    model.add(Activation("tanh"))
    
    return model

In [ ]:
def make_discriminator_model():
    model = tf.keras.Sequential()
    
    model.add(Conv2D(32,kernel_size=4, strides=2, padding="same",input_shape=[IMG_SIZE, IMG_SIZE, 3]))
    model.add(LeakyReLU(0.2))
    
    model.add(Dropout(0.25))
    
    model.add(Conv2D(64,kernel_size=4, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(0.2))
    
    model.add(Dropout(0.25))
    
    model.add(Conv2D(128,kernel_size=4, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(0.2))
    
    model.add(Dropout(0.25))
    
    model.add(Conv2D(256,kernel_size=4, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(0.2))
    
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(1,Activation("sigmoid")))
    
    return model

In [ ]:
generator=make_generator_model()
discriminator=make_discriminator_model()

In [ ]:
def save_images(cnt,noise):
  image_array = np.full(( 
      PREVIEW_MARGIN + (PREVIEW_ROWS * (IMG_SIZE+PREVIEW_MARGIN)), 
      PREVIEW_MARGIN + (PREVIEW_COLS * (IMG_SIZE+PREVIEW_MARGIN)), 3), 
      255, dtype=np.uint8)
  
  generated_images = generator.predict(noise)

  generated_images = 0.5 * generated_images + 0.5

  image_count = 0
  for row in range(PREVIEW_ROWS):
      for col in range(PREVIEW_COLS):
        r = row * (IMG_SIZE+16) + PREVIEW_MARGIN
        c = col * (IMG_SIZE+16) + PREVIEW_MARGIN
        image_array[r:r+IMG_SIZE,c:c+IMG_SIZE] = generated_images[image_count] * 255
        image_count += 1

          
  output_path = os.path.join(DATA_PATH,'output')
  if not os.path.exists(output_path):
    os.makedirs(output_path)
  
  filename = os.path.join(output_path,f"train-{cnt}.png")
  im = Image.fromarray(image_array)
  im.save(filename)

In [ ]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def get_generator_loss(generated_output):
    return tf.nn.sigmoid_cross_entropy_with_logits(labels = tf.ones_like(generated_output), logits = generated_output)
def get_discriminator_loss(generator_output, real_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(generator_output), generator_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [ ]:
generator_optimizer = tf.optimizers.Adam(1.5e-4)
discriminator_optimizer = tf.optimizers.Adam(1.5e-4)

In [ ]:
def train_step(images):
    seed = tf.random.normal([BATCH_SIZE, RANDOM_SEED])
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        gen_images = generator(seed,training=True)
        
        real_output = discriminator(images,training=True)
        fake_output = discriminator(gen_images,training=True)
        
        gen_loss = get_generator_loss(fake_output)
        disc_loss = get_discriminator_loss(fake_output,real_output)
        
        grad_of_generator = gen_tape.gradient(gen_loss,generator.trainable_variables)
        grad_of_discriminator = disc_tape.gradient(disc_loss,discriminator.trainable_variables)
        
        generator_optimizer.apply_gradients(zip(grad_of_generator,generator.trainable_variables))
        discriminator_optimizer.apply_gradients(zip(grad_of_discriminator,discriminator.trainable_variables))
        
    return gen_loss,disc_loss

In [ ]:
def train(images, epochs):
    fixed_seed = np.random.normal(0, 1, (PREVIEW_ROWS * PREVIEW_COLS, RANDOM_SEED))
    for epoch in range(epochs):
        gen_loss_list=[]
        disc_loss_list=[]
        for image in train_dataset:
            t = train_step(image)
            gen_loss_list.append(t[0])
            disc_loss_list.append(t[1])
        g_loss = sum(gen_loss_list)/len(gen_loss_list)
        d_loss = sum(disc_loss_list)/len(disc_loss_list)
        print (f'Epoch {epoch+1}, gen loss={g_loss},disc loss={d_loss}')

In [ ]:
train(train_dataset,EPOCHS)